In [549]:
import requests
from urllib.parse import urlparse
import json
import re
import os
import pandas as pd
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from tqdm import tqdm
from clickhouse_driver import Client
import base64

In [550]:
!pip install docker
!pip install clickhouse_driver
!docker run -d --name clickhouse-server --ulimit nofile=262144:262144 -p 9000:9000 yandex/clickhouse-server

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
docker: Error response from daemon: Conflict. The container name "/clickhouse-server" is already in use by container "9e0ea3c268de4eb50e84255ebbae6719aeec84d9e1712fb037c46816b6034544". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


In [551]:
from fake_useragent import UserAgent

url = 'https://books.toscrape.com/'
user = UserAgent()
headers = {
    "User-Agent":user.random,
}
params = {}
session = requests.session()
response = session.get(url=url, params=params, headers=headers)

In [552]:

import clickhouse_connect

In [553]:
client = Client(host='localhost',
                user='default',
                password='',
                port=9000
                )


try:
    result = client.execute('SHOW TABLES')
    print(result)
except Exception as e:
    print(f"Encountered an error: {e}")

[('trips',)]


In [554]:
def get_categories_from_response(response):
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, features='html.parser')
    else:
        print(response.status_code)
    url_categories = []
    result = soup.find_all(name="a", attrs={"href":re.compile("^catalogue/category/books/")})
    for i in result:
        url_categories.append(i.get("href"))
    url_categories_links = []
    for i in url_categories:
        url_categories_links.append(f'{url}{i}')
    return url_categories_links

url_categories_links = get_categories_from_response(response)

In [555]:
def get_url_from_row(url):
    result = []
    response = session.get(url=url, params=params, headers=headers)
    soup = BeautifulSoup(response.text, features='html.parser')
    data_page_article = soup.find(name="ol", attrs={"class":"row"}).find_all(name="h3")
    for card in data_page_article:
        result.append("/".join(url.split("/")[:4]+(card.find("a").get('href')).split("/")[-2:]))
    return result

def urls_from_paginator(urls):
        url_from_paginator = []
        for url in tqdm(urls):
                while url != None:
                        response = session.get(url=url, params=params, headers=headers)
                        soup = BeautifulSoup(response.text, features='html.parser')
                        if soup.find(name="li", attrs={"class":"next"}) is not None:
                                next_lin="/".join(url.split("/")[:-1]+[soup.find(name="li", attrs={"class":"next"}).findChild("a").get("href")])
                                url_from_paginator.append(next_lin)
                                url = next_lin
                        else:
                                url = None
        result = url_from_paginator
        return result

In [556]:
urls_paginator = urls_from_paginator(url_categories_links)

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:12<00:00,  3.92it/s]


In [557]:
print(len(urls_paginator))
print(len(url_categories_links))
url_categories_links.extend(urls_paginator)
print(len(url_categories_links))

30
50
80


In [558]:
def all_books_links(urls):
    finall_all_links = []
    for url in tqdm(url_categories_links):
        link_from_row = get_url_from_row(url)
        finall_all_links.extend(link_from_row)
    return finall_all_links

In [559]:
boks_links = all_books_links(url_categories_links)

100%|██████████| 80/80 [00:12<00:00,  6.26it/s]


In [560]:
def data_extraxt(url):
    result = {}
    response = session.get(url=url, params=params, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, features='html.parser')
    else:
        print(response.status_code)

    card = soup.find(name="div", attrs={"class":"product_main"})
    if card:
        result['name'] = card.find(name="h1").text
        price = card.find(name="p", attrs={"class":"price_color"}).text
        result['price'] = float(price[2:])
        count = card.find(name="p", attrs={"class":"instock availability"}).text
        pattern = r'(\d{1,9})'
        count = int(re.search(pattern,count).group())
        result['count'] = count
    descriptions = soup.find(name="article", attrs={"class":"product_page"}).find(name="p", attrs={'class': False, 'id': False})
    if descriptions:
        result['description'] = descriptions.text
    return result

In [561]:
columns = ['name','price', 'count', 'description']
result_data = pd.DataFrame(columns = columns)

In [562]:
client.execute(
        '''
        CREATE TABLE IF NOT EXISTS book
        (
            name String,
            price String,
            count String,
            description String
        )
        ENGINE = MergeTree()
        ORDER BY name;
        '''
        ) 

[]

In [563]:
columns = ['name','price', 'count', 'description']
result_data = pd.DataFrame(columns = columns)

In [564]:
for index, link in enumerate(tqdm(boks_links)):
    data_page = data_extraxt(link)
    result_data.loc[index] = data_page.get('name'), data_page.get('price'), data_page.get('count'), data_page.get('description')

100%|██████████| 1000/1000 [02:34<00:00,  6.46it/s]


In [565]:
import base64

def code_dase64(text):
    try:
        if text is None:
            text = 'None'
        if type(text)!='str':
            return base64.b64encode(str(text).encode("utf-8")).hex()
        return base64.b64encode(text.encode("utf-8")).hex()
    except:
        print(f"error convert data{text}")


In [566]:
for index, link in enumerate(tqdm(boks_links)):
    data_page = data_extraxt(link)
    name = code_dase64(data_page.get('name'))
    price = code_dase64(data_page.get('price'))
    count = code_dase64(data_page.get('count'))
    description = code_dase64(data_page.get('description'))
    result = (name, price,count, description)
    sql = f'''INSERT INTO book (*) VALUES ('{result[0]}','{result[1]}','{result[2]}','{result[3]}')'''
    try:
        client.execute(sql)
    except:
        print(f'error save {result}')

100%|██████████| 1000/1000 [02:39<00:00,  6.28it/s]


In [567]:
result_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         1000 non-null   object 
 1   price        1000 non-null   float64
 2   count        1000 non-null   int64  
 3   description  998 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 39.1+ KB


In [568]:
result_data['name'] = result_data['name'].astype('str')
result_data['description'] = result_data['description'].astype('str')
result_data['price'] = result_data['price'].astype('float64')
result_data['count'] = result_data['count'].astype('int64')

In [569]:
client.execute('SHOW TABLES')

[('book',), ('trips',)]

In [570]:
client.execute('SELECT count() FROM book')

[(1000,)]